In [47]:
import pandas as pd

# ✅ 1. 데이터 불러오기
df_review = pd.read_json("data/review_0503.json", lines=True)
df_user = pd.read_json("data/user_0503.json", lines=True)
df_biz = pd.read_json("data/business.json", lines=True)

In [49]:
import pandas as pd

# 1. 데이터 불러오기
df_review = pd.read_json("data/review_0503.json", lines=True)
df_user = pd.read_json("data/user_0503.json", lines=True)
df_biz = pd.read_json("data/business.json", lines=True)

# 2. 컬럼명 충돌 방지
df_user = df_user.rename(columns={"name": "user_name"})
df_biz = df_biz.rename(columns={"name": "business_name"})

# 3. 사용자 정보 병합
df_merged = df_review.merge(df_user, on="user_id", how="left")

# 4. 음식점 정보 병합
df_merged = df_merged.merge(df_biz, on="business_id", how="left")

# 5. 결측치 개수 출력
na_count = df_merged.isna().sum()
na_cols = na_count[na_count > 0]
print("📊 [결측치 있는 컬럼]")
print(na_cols.sort_values(ascending=False))

# 6. 병합 완료 정보
print(f"\n✅ 병합된 데이터 수: {len(df_merged):,}개")
print(df_merged.info())

📊 [결측치 있는 컬럼]
user_name         272580
review_count_x    272580
useful_y          272580
average_stars     272580
dtype: int64

✅ 병합된 데이터 수: 676,701개
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676701 entries, 0 to 676700
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   review_id       676701 non-null  object        
 1   user_id         676701 non-null  object        
 2   business_id     676701 non-null  object        
 3   stars_x         676701 non-null  int64         
 4   useful_x        676701 non-null  int64         
 5   date            676701 non-null  datetime64[ns]
 6   sentiment       676701 non-null  object        
 7   cleaned_text    676701 non-null  object        
 8   token_length    676701 non-null  int64         
 9   user_name       404121 non-null  object        
 10  review_count_x  404121 non-null  float64       
 11  useful_y        404121 non-null  float64     

In [51]:
import numpy as np

# 1. 사용자별 평균, 표준편차 계산
user_stats = df_merged.groupby("user_id")["stars_x"].agg(["mean", "std"]).reset_index()
user_stats.columns = ["user_id", "user_mean", "user_std"]
user_stats["user_std"] = user_stats["user_std"].fillna(0.001)  # std=0 방지

# 2. 다시 병합
df_merged = df_merged.merge(user_stats, on="user_id", how="left")

# 3. z-score 계산
df_merged["z_score"] = (df_merged["stars_x"] - df_merged["user_mean"]) / df_merged["user_std"]

# 4. 감성 점수 생성 함수
def score_from_z(z):
    if z > 0.5:
        return 2  # 긍정
    elif z < -0.5:
        return 0  # 부정
    else:
        return 1  # 중립

df_merged["sentiment_score"] = df_merged["z_score"].apply(score_from_z)

# 5. 확인
print(df_merged[["user_id", "stars_x", "user_mean", "z_score", "sentiment_score"]].head())

                  user_id  stars_x  user_mean   z_score  sentiment_score
0  _7bHUi9Uuf5__HHc_Q8guQ        5   5.000000  0.000000                1
1  eUta8W_HdHMXPzLBBZhL1A        1   1.000000  0.000000                1
2  smOvOajNG0lS4Pq7d8g4JQ        4   3.421053  0.523441                2
3  Dd1jQj7S-BFGqRbApFzCFw        5   5.000000  0.000000                1
4  IQsF3Rc6IgCzjVV9DE8KXg        5   3.388060  1.381070                2


In [56]:
# 🔥 모든 결측치가 하나라도 있는 행 제거
df_clean = df_merged.dropna().reset_index(drop=True)

print(f"🧹 결측치 제거 후 남은 행 수: {len(df_clean):,}개")

🧹 결측치 제거 후 남은 행 수: 397,082개


In [57]:
df_clean = df_clean.rename(columns={
    "stars_x": "user_rating",            # 사용자가 준 평점
    "useful_x": "review_useful",         # 리뷰에 대한 유용 투표 수
    "review_count_x": "user_review_count",  # 사용자 총 리뷰 수
    "useful_y": "user_total_useful",     # 사용자가 받은 유용 투표 총합
    "stars_y": "business_avg_rating",    # 음식점 평균 평점
    "review_count_y": "business_review_count",  # 음식점 리뷰 수
    "stars_y": "business_avg_rating",    # 음식점 평점
    "z_score": "user_rating_zscore",     # 사용자 기준 평점 표준화
    "sentiment_score": "sentiment_label" # 0=부정, 1=중립, 2=긍정
})

In [82]:
# address가 너무 짧은 경우, NaN으로 처리 (예: len <= 3)
df_clean.loc[df_clean["address"].str.len() <= 3, "address"] = pd.NA

# 그런 다음 완전 결측치 제거 (선택적)
df_clean = df_clean.dropna(subset=["address"]).reset_index(drop=True)

In [84]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396098 entries, 0 to 396097
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   review_id              396098 non-null  object        
 1   user_id                396098 non-null  object        
 2   business_id            396098 non-null  object        
 3   user_rating            396098 non-null  int64         
 4   review_useful          396098 non-null  int64         
 5   date                   396098 non-null  datetime64[ns]
 6   sentiment              396098 non-null  object        
 7   cleaned_text           396098 non-null  object        
 8   token_length           396098 non-null  int64         
 9   user_name              396098 non-null  object        
 10  user_review_count      396098 non-null  float64       
 11  user_total_useful      396098 non-null  float64       
 12  average_stars          396098 non-null  floa

In [85]:
df_clean.to_csv("data/merged_0503.csv", index=False)
print("✅ 감성 점수 포함된 파일 저장 완료: data/merged_0503.csv")

✅ 감성 점수 포함된 파일 저장 완료: data/merged_0503.csv


In [87]:
import pandas as pd
from transformers import AutoTokenizer
from tqdm import tqdm

# 1. 데이터 로드
df = pd.read_csv("data/merged_0503.csv")

# 2. DeBERTa 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

# 3. 결과 저장 리스트
input_ids_list = []
attention_mask_list = []

# 4. 텍스트 인코딩 (max_length=128)
for text in tqdm(df["cleaned_text"], desc="🧠 Tokenizing"):
    encoded = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    input_ids_list.append(encoded["input_ids"])
    attention_mask_list.append(encoded["attention_mask"])

# 5. 결과 DataFrame에 추가
df["input_ids"] = input_ids_list
df["attention_mask"] = attention_mask_list

# 6. JSONL 파일로 저장 (라인별 객체 구조)
df.to_json("data/merged_0503_tokenized.json", orient="records", lines=True)

print("✅ 저장 완료: data/merged_0503_tokenized.json")

/opt/anaconda3/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
🧠 Tokenizing: 100%|██████████| 396098/396098 [00:58<00:00, 6733.98it/s]


✅ 저장 완료: data/merged_0503_tokenized.json


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396098 entries, 0 to 396097
Data columns (total 29 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   review_id              396098 non-null  object        
 1   user_id                396098 non-null  object        
 2   business_id            396098 non-null  object        
 3   user_rating            396098 non-null  int64         
 4   review_useful          396098 non-null  int64         
 5   date                   396098 non-null  datetime64[ns]
 6   sentiment              396098 non-null  object        
 7   cleaned_text           396098 non-null  object        
 8   token_length           396098 non-null  int64         
 9   user_name              396098 non-null  object        
 10  user_review_count      396098 non-null  int64         
 11  user_total_useful      396098 non-null  int64         
 12  average_stars          396098 non-null  floa

In [8]:
import pandas as pd

# 1. 토큰화된 전체 데이터 로드
df = pd.read_json("data/merged_0503_tokenized.json", lines=True)

# 2. 감성 분석용 데이터셋 생성
sentiment_cols = [
    "review_id", "user_id", "business_id",
    "cleaned_text", "user_rating", "sentiment_label",
    "input_ids", "attention_mask"
]
df_sentiment = df[sentiment_cols]
df_sentiment.to_json("data/sentiment_data.json", orient="records", lines=True)
print("✅ 저장 완료: data/sentiment_data.json")

# 3. 추천 시스템용 데이터셋 생성
recommend_cols = [
    "user_id", "business_id", "user_rating", "sentiment_label"
]
df_recommend = df[recommend_cols]
df_recommend.to_json("data/recommend_data.json", orient="records", lines=True)
print("✅ 저장 완료: data/recommend_data.json")

✅ 저장 완료: data/sentiment_data.json
✅ 저장 완료: data/recommend_data.json
